<font color= "green">Here the mother computer acts as a hub, collecting data from FED and then sending commands or events to the Arduino board.
Arduino receives those event markers from the mother computer and translates them into digital pulses for the TDT system</font>

## The data flow that we need to sync FEDs with TDT:
### FED3 to Computer: The FED3 devices send behavioral event data to the computer through their respective serial connections.
### Computer to Arduino: The computer processes this data and forwards relevant event information to the Arduino through a separate serial connection.
### Arduino to TDT System: The Arduino generates digital pulses or signals in response to the data it receives. These pulses are sent to the TDT system to mark the timing of behavioral events, enabling synchronization with neural recordings.

In [ ]:
# this code should send data to Google spreadsheet and at the same time to Arduino board

import serial
import threading
import datetime  
import gspread
from google.oauth2.service_account import Credentials

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Setup Google Sheets
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Path to your downloaded JSON file
CREDS_FILE = r"C:\Users\hta031\Github\HOME_PHOTOMETRY\scripts\Python_codes\homephotometry-102601e8b10f.json"

creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
client = gspread.authorize(creds)

# Google Spreadsheet ID
SPREADSHEET_ID = "1oybqWp_7b9_oiR-a1Xy0YLw8LwvGfqtmSz2lYfEzrBk"

def get_or_create_worksheet(spreadsheet, title):
    try:
        # Try to open the worksheet by title
        sheet = spreadsheet.worksheet(title)
        print(f"Worksheet '{title}' found.")
    except gspread.exceptions.WorksheetNotFound:
        # If the worksheet is not found, create it
        print(f"Worksheet '{title}' not found. Creating a new one.")
        sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
        sheet.append_row(column_headers)
    return sheet

def send_to_arduino(data, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    arduino_ser.write(data.encode('utf-8'))
    print(f"Sent to Arduino: {data}")

def read_from_port(ser, worksheet_name, arduino_ser):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        # to sync all the FED units with each other, I tend not to rely on FED clock,
        # Ignore the first field (timestamp from FED device) and use the computer's timestamp
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        # Assuming the data matches the order of the remaining column_headers
        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Append the row to Google Sheet
            sheet.append_row([timestamp] + data_list)
            
            # Send relevant data to Arduino
            event_type = data_list[9]  # Example: using the 10th field as the event identifier
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM17"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    worksheet_name = f"Port_{port}"  # Create a unique worksheet name for each port
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, worksheet_name, arduino_ser)).start()


In [ ]:
int pulsePin = 7; // Digital pin for output pulse

void setup() {
  Serial.begin(115200);
  pinMode(pulsePin, OUTPUT);
}

void loop() {
  if (Serial.available() > 0) {
    String data = Serial.readStringUntil('\n');
    // Generate a pulse when data is received
    digitalWrite(pulsePin, HIGH);
    delay(10);  // 10 ms pulse
    digitalWrite(pulsePin, LOW);
    
    // Optionally, you can parse `data` for more specific actions
    Serial.print("Received: ");
    Serial.println(data);
  }
}
